# 基本情况
比赛：2024中国高校计算机大赛 — 大数据挑战赛（BDC2024） 基于气象大数据的自动站实况联合预测

成绩：

|阶段|排名|分数|
|:---:|:---:|:---:|
|初赛 A 榜|1|0.9238588370641594|
|初赛 B 榜|7|0.9763797357608868|
|复赛 A 榜|17|1.1917952887389343|
|复赛 B 榜|21|1.1687667345811001|
|复现|19|-|

# 代码说明  

## 环境配置  
注明使用镜像，包括python、pytorch等依赖的版本  
如果是自定义镜像，需要注明基础镜像的名称以及额外安装包的名称、版本并按照“队伍名称+复现镜像”格式修改命名  

Avid_复现镜像  
基础镜像：Torch 2.2.2 Cuda 12.1 Python 3.11.8 Ubuntu 22.04  
其他安装的包：
- einops==0.8.0  
- pykalman==0.9.7  
- reformer_pytorch==1.4.4  

线下使用的环境在`BDC.yaml`中可以看到。

由于最开始平台离线训练有些问题，所以我采用线上notebook训练。  同时为了节省训练时间，我选择使用nohup后台并行训练。  

## 算法  

我的模型比较简单粗暴，根据输入序列直接预测3天的时间序列，并且做了很多融模。模型主要包含了 iTransformer 和 iFlashFormer 两个模型的 encoder，并使用 MMoE 做了融合，在 decoder 中包含了 BiLSTM。  

### 整体思路介绍  

根据输入的 7 天历史数据（包括 temp、wind 以及 36 个协变量）来预测未来 3 天的 temp 和 wind。  

### 数据处理  

由于数据中包含了全 0 站点以及大量缺失数据，因此我先将全 0 的站点删除，然后根据采样的时间窗口（168 + 72）中等于众数占比较高的窗口以及窗口内方差过高过低的 temp、wind 数据全部删除。  

异常数据示例：  


![Image Name](asset/si59xdos3h.png)  


```python  
def judge(seq_x, seq_y):  
    # seq_x [168, 38], seq_y [72, 2]  
    data_temp = np.concatenate([seq_x[:, -2], seq_y[:, -2]], axis=0)  
    v = data_temp.var()  
    if v < 2.7 or v > 43.2:  
        return False  
    a = stats.mode(data_temp, keepdims=True)[0][0]  
    if (data_temp == a).sum() / 240 > 0.1745:  
        return False  

    data_wind = np.concatenate([seq_x[:, -1], seq_y[:, -1]], axis=0)  
    v = data_wind.var()  
    if v < 0.59 or v > 12.2:  
        return False  
    a = stats.mode(data_wind, keepdims=True)[0][0]  
    if (data_wind == a).sum() / 240 > 0.373:  
        return False  
    return True  
```
其中的固定浮点数是根据线下数据分析得到的（方差保留 5%\~95% 的数据，众数占比保留 0\~90% 的数据）。  

在全部的时间窗口中（约 6000w），我将时间窗口（240h）内的方差，以及值等于众数占比的分布情况做了可视化如下。  

![stat](asset/si59vifhxo.png)  

根据实验（其中的 $VAL 表示对应的值）：  
> - temp:  
>   - 5%~95% var: 2.7 < $VAL < 43.2  
>   - 0~90% mode: $VAL < 0.1745  
> - wind:  
>   - 5%~95% var: 0.59 < $VAL < 12.2  
>   - 0~90% mode: $VAL < 0.373  

由于协变量 era5 是 3 小时一次，所以对数据进行增广，使用的方法并不是标准的线性插值，因为那样太平滑，在初赛中尝试的效果并不好，因此我的做法是：先按照 baseline repeat 扩充到正确维度，然后：  
```python  
# batch_x [bs, 168, 38]  
for idx in range(0, 165, 3):  
    temp_a = (batch_x[:, idx, :-2] + batch_x[:, idx + 3, :-2]) / 3  
    batch_x[:, idx + 1, :-2] = temp_a  
    batch_x[:, idx + 2, :-2] = 2 * temp_a  
```
这样做可以在相邻两个值变化不大时增加震荡幅度，而在相邻两个之变化较大时偏向平滑（$a_{i} = 2 * a_{i+3}$ （或 $2 * a_{i} = a_{i+3}$） 为临界点，将插值在 $[a_{i},a_{i+3}]$ （或 $[a_{i+3},a_{i}]$） 区间内/外两种情况分开，其中 $a\in\mathbb{R}^{168}$ 为单时间序列，且 $i\in \text{range}(0, 165, 3)$）。  

然后增加特征维度，我这里增加的特征维度主要是对 temp 和 wind 分别进行 diff、roll、九个站点协变量的均值、协变量中两个风速基向量的矢量和等，加上 temp 和 wind 一共 56 个特征。  

### 网络结构  

网络结构整体上是 follow 男枪哥的（感谢），两个不同的 encoder (iTransformer + iFlashFormer) 作为 MMoE 的两个 expert，非常简单的结构。  


![Image Name](asset/si59wroydx.png)  

### 损失函数  

考虑到 MSE 的平滑作用，我在损失函数中加上了 MAE 损失，整体 loss 中 MSE 占比 0.7，MAE 占比 0.3。  


### 数据扩增  

在训练阶段，加上了高斯噪声，其中均值为 0，标准差为历史窗口内标准差的 0.1 倍。  

```python  
stds = torch.std(batch_x, dim=1, keepdim=True) * 0.1  
batch_x = batch_x + torch.normal(0.0, stds)  
```

在 index.py 中的测试阶段，使用了 TTA，多次对历史数据加上窗口内标准差的 0.1 倍然后求均值。具体见 [index.py#L147](index.py#L147) 等。  

### 模型集成  

模型集成部分主要采取了 5 种集成策略：  

- Epoch 集成：每一个 epoch 训练结束后的 checkpoint 作为融合的一个 item（考虑到与 EMA 的作用类似，我也没有在训练代码中使用 EMA）；  
- Fold 集成：划分数据时我先按照站点划分，取 95% 的站点作为线下训练数据，剩下 5% 的数据作为本地 cv。在那 95% 的训练数据中，使用 K-Fold 交叉（K=3），对着 95% 站点中的所有时间窗口分割 Fold，并取每一折中的 1/3 部分（而非 2/3 部分）作为训练数据；
- Seed 集成：考虑到不同种子会得到不同的 95% 训练站点以及时间窗口集合，因此我在实验中使用了多个种子依次训练；
- Task 集成：分别训练了预测 temp、预测 wind 以及同时预测 temp/wind 的模型； 
- 数据清洗集成：分别用清洗后的数据和原始数据训练。

所以一共有 $3*3*2*3*2=108$ 个 checkpoint。（大力出奇迹）

上面所有产生的 checkpoint 都会在 index.py 中使用并进行 forecasting，最终结果取均值。  

### 算法的其他细节  

对 iTransformer attention 的 key 和 value 加上了 RoPE ([SelfAttention_Family.py#L68](my_code/layers/SelfAttention_Family.py#L68))，使用了 PGD 对抗训练以及 torch.cuda.amp 混合精度，学习率按照 baseline 方法每一个 epoch 缩小一半。  

## 训练和测试流程 
一键运行`train.ipynb`或者分别运行 `train.sh` 和`train_clean.sh`开始，详见代码。  

## 计算资源

线下计算资源：Intel Xeon Silver 4310 CPU @ 2.10GHz, 2 * RTX 4090 24GB, RAM 256GB， SWAP 64GB

线上训练资源：腾讯云双卡 V100 的算力（显存 32GB * 2，CPU 18核 80G），限时 72 小时

线上推理资源：单卡 V100 32GB

## 耗时

训练：线下大约 48 小时，线上大约 56 小时。
评估：大约 15 min。
